In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM
from tensorflow.keras.regularizers import l2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.metrics import *

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[3], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation= 'relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [8]:
def summarize_results(accuracies,precisions,recalls,aucs):
    m, s = mean(accuracies), std(accuracies)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(precisions), std(precisions)
    print( ' Precision: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(recalls), std(recalls)
    print( ' Recall: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(aucs), std(aucs)
    print( ' AUC: %.3f%% (+/-%.3f) ' % (m, s))

In [9]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [10]:
run_experiment(X,y)

Epoch 1/30
58/58 [==============================] - 9s 161ms/step - loss: 0.6904 - accuracy: 0.5137 - precision: 0.4758 - recall: 0.3668 - auc: 0.5332
Epoch 2/30
58/58 [==============================] - 10s 173ms/step - loss: 0.6143 - accuracy: 0.6681 - precision: 0.6551 - recall: 0.6168 - auc: 0.7273
Epoch 3/30
58/58 [==============================] - 10s 173ms/step - loss: 0.5655 - accuracy: 0.7054 - precision: 0.6732 - recall: 0.7220 - auc: 0.7718
Epoch 4/30
58/58 [==============================] - 10s 176ms/step - loss: 0.5322 - accuracy: 0.7755 - precision: 0.7767 - recall: 0.7313 - auc: 0.8223
Epoch 5/30
58/58 [==============================] - 10s 174ms/step - loss: 0.5750 - accuracy: 0.7196 - precision: 0.7038 - recall: 0.6939 - auc: 0.7720
Epoch 6/30
58/58 [==============================] - 10s 173ms/step - loss: 0.5769 - accuracy: 0.7010 - precision: 0.7089 - recall: 0.6145 - auc: 0.7727
Epoch 7/30
58/58 [==============================] - 10s 173ms/step - loss: 0.5107 - accur

58/58 [==============================] - 10s 179ms/step - loss: 0.1979 - accuracy: 0.9266 - precision_1: 0.9227 - recall_1: 0.9206 - auc_1: 0.9748
Epoch 24/30
58/58 [==============================] - 10s 179ms/step - loss: 0.2322 - accuracy: 0.9157 - precision_1: 0.8857 - recall_1: 0.9416 - auc_1: 0.9639
Epoch 25/30
58/58 [==============================] - 10s 180ms/step - loss: 0.2156 - accuracy: 0.9179 - precision_1: 0.8931 - recall_1: 0.9369 - auc_1: 0.9695
Epoch 26/30
58/58 [==============================] - 10s 180ms/step - loss: 0.1481 - accuracy: 0.9573 - precision_1: 0.9451 - recall_1: 0.9650 - auc_1: 0.9830
Epoch 27/30
58/58 [==============================] - 10s 179ms/step - loss: 0.2450 - accuracy: 0.9343 - precision_1: 0.9381 - recall_1: 0.9206 - auc_1: 0.9621
Epoch 28/30
58/58 [==============================] - 10s 180ms/step - loss: 0.2747 - accuracy: 0.9091 - precision_1: 0.8984 - recall_1: 0.9089 - auc_1: 0.9465
Epoch 29/30
58/58 [==============================] - 10s 1

58/58 [==============================] - 11s 182ms/step - loss: 0.3028 - accuracy: 0.8806 - precision_3: 0.8521 - recall_3: 0.9019 - auc_3: 0.9442
Epoch 15/30
58/58 [==============================] - 10s 180ms/step - loss: 0.2894 - accuracy: 0.8938 - precision_3: 0.8770 - recall_3: 0.8995 - auc_3: 0.9465
Epoch 16/30
58/58 [==============================] - 10s 181ms/step - loss: 0.3075 - accuracy: 0.8795 - precision_3: 0.8732 - recall_3: 0.8692 - auc_3: 0.9406
Epoch 17/30
58/58 [==============================] - 10s 179ms/step - loss: 0.2681 - accuracy: 0.9047 - precision_3: 0.9069 - recall_3: 0.8879 - auc_3: 0.9518
Epoch 18/30
58/58 [==============================] - 10s 179ms/step - loss: 0.2846 - accuracy: 0.8905 - precision_3: 0.8534 - recall_3: 0.9252 - auc_3: 0.9522
Epoch 19/30
58/58 [==============================] - 10s 179ms/step - loss: 0.1922 - accuracy: 0.9277 - precision_3: 0.9229 - recall_3: 0.9229 - auc_3: 0.9753
Epoch 20/30
58/58 [==============================] - 10s 1

In [11]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(n_timesteps,n_features)))
    model.add(LSTM(50))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation= 'relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [12]:
def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_2(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [13]:
run_experiment_2(X,y)

Epoch 1/30
58/58 [==============================] - 18s 314ms/step - loss: 0.6693 - accuracy: 0.5531 - precision_5: 0.5394 - recall_5: 0.3201 - auc_5: 0.6095
Epoch 2/30
58/58 [==============================] - 18s 314ms/step - loss: 0.5418 - accuracy: 0.7481 - precision_5: 0.7152 - recall_5: 0.7687 - auc_5: 0.8039
Epoch 3/30
58/58 [==============================] - 18s 311ms/step - loss: 0.4271 - accuracy: 0.8204 - precision_5: 0.8128 - recall_5: 0.8014 - auc_5: 0.8838
Epoch 4/30
58/58 [==============================] - 18s 317ms/step - loss: 0.4100 - accuracy: 0.8445 - precision_5: 0.8295 - recall_5: 0.8411 - auc_5: 0.8880
Epoch 5/30
58/58 [==============================] - 18s 316ms/step - loss: 0.4154 - accuracy: 0.8226 - precision_5: 0.8674 - recall_5: 0.7336 - auc_5: 0.8898
Epoch 6/30
58/58 [==============================] - 18s 317ms/step - loss: 0.3442 - accuracy: 0.8708 - precision_5: 0.8744 - recall_5: 0.8458 - auc_5: 0.9242
Epoch 7/30
58/58 [==============================] - 

58/58 [==============================] - 18s 317ms/step - loss: 0.1427 - accuracy: 0.9540 - precision_6: 0.9468 - recall_6: 0.9556 - auc_6: 0.9828
Epoch 23/30
58/58 [==============================] - 18s 315ms/step - loss: 0.1929 - accuracy: 0.9441 - precision_6: 0.9314 - recall_6: 0.9509 - auc_6: 0.9714
Epoch 24/30
58/58 [==============================] - 18s 317ms/step - loss: 0.2366 - accuracy: 0.9168 - precision_6: 0.9000 - recall_6: 0.9252 - auc_6: 0.9611
Epoch 25/30
58/58 [==============================] - 18s 318ms/step - loss: 0.1493 - accuracy: 0.9617 - precision_6: 0.9538 - recall_6: 0.9650 - auc_6: 0.9805
Epoch 26/30
58/58 [==============================] - 18s 317ms/step - loss: 0.1067 - accuracy: 0.9715 - precision_6: 0.9610 - recall_6: 0.9790 - auc_6: 0.9881
Epoch 27/30
58/58 [==============================] - 18s 316ms/step - loss: 0.1706 - accuracy: 0.9496 - precision_6: 0.9401 - recall_6: 0.9533 - auc_6: 0.9764
Epoch 28/30
58/58 [==============================] - 18s 3

58/58 [==============================] - 18s 316ms/step - loss: 0.2520 - accuracy: 0.8981 - precision_8: 0.9056 - recall_8: 0.8738 - auc_8: 0.9615
Epoch 14/30
58/58 [==============================] - 18s 315ms/step - loss: 0.2385 - accuracy: 0.9135 - precision_8: 0.9068 - recall_8: 0.9089 - auc_8: 0.9640
Epoch 15/30
58/58 [==============================] - 18s 316ms/step - loss: 0.2521 - accuracy: 0.9102 - precision_8: 0.9005 - recall_8: 0.9089 - auc_8: 0.9593
Epoch 16/30
58/58 [==============================] - 18s 311ms/step - loss: 0.1876 - accuracy: 0.9354 - precision_8: 0.9184 - recall_8: 0.9463 - auc_8: 0.9757
Epoch 17/30
58/58 [==============================] - 18s 312ms/step - loss: 0.1349 - accuracy: 0.9562 - precision_8: 0.9470 - recall_8: 0.9603 - auc_8: 0.9866
Epoch 18/30
58/58 [==============================] - 18s 315ms/step - loss: 0.2706 - accuracy: 0.9025 - precision_8: 0.8792 - recall_8: 0.9182 - auc_8: 0.9545
Epoch 19/30
58/58 [==============================] - 18s 3